In [7]:
import numpy as np
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
intents_file = open('intents.txt').read()
intents = json.loads(intents_file)

In [8]:
words=[]
classes = []
documents = []
ignore_letters = ['!', '?', ',', '.']
for intent in intents['intents']:
    for pattern in intent['patterns']:
     #tokenize each word
        word = nltk.word_tokenize(pattern)
        words.extend(word)        
     #add documents in the corpus
        documents.append((word, intent['tag']))
     # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [9]:
documents

[(['Salut'], 'bonjour'),
 (['Bonjour'], 'bonjour'),
 (['Hey'], 'bonjour'),
 (['Yo'], 'bonjour'),
 (['Hello'], 'bonjour'),
 (['hi'], 'bonjour'),
 (['bonsoir'], 'bonjour'),
 (['bjr'], 'bonjour'),
 (['bsr'], 'bonjour'),
 (['question'], 'question'),
 (['besoin'], 'question'),
 (['problème'], 'question'),
 (['interrogation'], 'question'),
 (['Bye'], 'au revoir'),
 (['prochaine'], 'au revoir'),
 (['revoir'], 'au revoir'),
 (['Adieu'], 'au revoir'),
 (['A+'], 'au revoir'),
 (['ciao'], 'au revoir'),
 (['Tchao'], 'au revoir'),
 (['xau'], 'au revoir'),
 (['Merci'], 'merci'),
 (['parfait'], 'merci'),
 (['aidé'], 'merci'),
 (['remercie'], 'merci'),
 (['sympa'], 'merci'),
 (['vitesse'], 'VMA'),
 (['VMA'], 'VMA'),
 (['km/h'], 'VMA'),
 (['aérobie'], 'VMA'),
 (['pouls'], 'heartbeat'),
 (['cardiaque'], 'heartbeat'),
 (['coeur'], 'heartbeat'),
 (['essouflement'], 'heartbeat'),
 (['essouflé'], 'heartbeat'),
 (['battements'], 'heartbeat'),
 (['respiration'], 'heartbeat'),
 (['distance'], 'steps'),
 (['par

In [10]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_letters]
words = sorted(list(set(words)))
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

### Création des données d'entrainements

In [11]:
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for word in words:
        bag.append(1) if word in pattern_words else bag.append(0)
        
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:23: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [12]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/200
16/16 [==============================] - 1s 2ms/step - loss: 2.5762 - accuracy: 0.1266 
Epoch 2/200
16/16 [==============================] - 0s 2ms/step - loss: 2.5505 - accuracy: 0.0759
Epoch 3/200
16/16 [==============================] - 0s 2ms/step - loss: 2.5350 - accuracy: 0.1139
Epoch 4/200
16/16 [==============================] - 0s 2ms/step - loss: 2.5295 - accuracy: 0.1013
Epoch 5/200
16/16 [==============================] - 0s 2ms/step - loss: 2.5405 - accuracy: 0.1139
Epoch 6/200
16/16 [==============================] - 0s 2ms/step - loss: 2.4911 - accuracy: 0.1392
Epoch 7/200
16/16 [==============================] - 0s 2ms/step - loss: 2.4727 - accuracy: 0.1772
Epoch 8/200
16/16 [==============================] - 0s 3ms/step - loss: 2.4895 - accuracy: 0.1392
Epoch 9/200
16/16 [==============================] - 0s 3ms/step - loss: 2.4572 - accuracy: 0.2278
Epoch 10/200
16/16 [==============================] - 0s 2ms/step - loss: 2.4346 - accuracy: 0.1519
Epoch 11

16/16 [==============================] - 0s 3ms/step - loss: 0.1739 - accuracy: 0.9494
Epoch 83/200
16/16 [==============================] - 0s 2ms/step - loss: 0.3876 - accuracy: 0.8734
Epoch 84/200
16/16 [==============================] - 0s 2ms/step - loss: 0.2854 - accuracy: 0.8734
Epoch 85/200
16/16 [==============================] - 0s 2ms/step - loss: 0.2441 - accuracy: 0.9241
Epoch 86/200
16/16 [==============================] - 0s 2ms/step - loss: 0.2063 - accuracy: 0.9241
Epoch 87/200
16/16 [==============================] - 0s 2ms/step - loss: 0.1974 - accuracy: 0.8987
Epoch 88/200
16/16 [==============================] - 0s 1ms/step - loss: 0.1850 - accuracy: 0.9494
Epoch 89/200
16/16 [==============================] - 0s 2ms/step - loss: 0.1991 - accuracy: 0.9367
Epoch 90/200
16/16 [==============================] - 0s 2ms/step - loss: 0.2383 - accuracy: 0.9114
Epoch 91/200
16/16 [==============================] - 0s 2ms/step - loss: 0.2650 - accuracy: 0.8987
Epoch 92/200


16/16 [==============================] - 0s 2ms/step - loss: 0.1575 - accuracy: 0.9494
Epoch 164/200
16/16 [==============================] - 0s 2ms/step - loss: 0.0694 - accuracy: 0.9873
Epoch 165/200
16/16 [==============================] - 0s 2ms/step - loss: 0.1782 - accuracy: 0.9494
Epoch 166/200
16/16 [==============================] - 0s 2ms/step - loss: 0.1379 - accuracy: 0.9367
Epoch 167/200
16/16 [==============================] - 0s 2ms/step - loss: 0.1187 - accuracy: 0.9494
Epoch 168/200
16/16 [==============================] - 0s 2ms/step - loss: 0.0869 - accuracy: 0.9747
Epoch 169/200
16/16 [==============================] - 0s 3ms/step - loss: 0.2025 - accuracy: 0.9494
Epoch 170/200
16/16 [==============================] - 0s 3ms/step - loss: 0.0543 - accuracy: 0.9620
Epoch 171/200
16/16 [==============================] - 0s 2ms/step - loss: 0.1786 - accuracy: 0.9494
Epoch 172/200
16/16 [==============================] - 0s 2ms/step - loss: 0.0892 - accuracy: 0.9747
Epoc